# Mean-Variance and Tracking Error Variance Analysis

## Load Libraries and Data

In [ ]:
# Load required libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from datetime import datetime

# Load scikit-learn library (LinearRegression only)
from sklearn.linear_model import LinearRegression

In [ ]:
# Load data
close_prices = pd.read_csv('EUROSTOXX50DataV2.csv', index_col = 0)
Symbols = close_prices.columns

# Number of calendar days between first and last data points in close_prices
first_date = datetime.strptime(close_prices.index.min(), '%Y-%m-%d')
last_date = datetime.strptime(close_prices.index.max(), '%Y-%m-%d')
n_cal_days = (last_date - first_date).days


## Compute the Mean and Covariance of (Annualized) Returns

We first assume that the risk-free asset has an annualized return of 2.5\% (i.e. $r_0 = 0.025$).

**Note:** As shall be seen below, the assumed quantity satisfies the condition $r_0 < \frac{b}{a}$, the latter quantity being dependent on the data at hand.

In [ ]:
# Risk-free rate (annualized)
r0 = 0.025

In [ ]:
# Annualization factor
ann_factor = 1 / ((n_cal_days / 365) / len(close_prices))

Note that the annualization factor is close to 252, which is the usually assumed number of trading days in a calendar year.

Annualization here is useful only for ''reading'' the expected return and the standard deviation of returns. The estimates are usually very small numbers, which are cumbersome to interpret/read, so we refer to their annualized versions instead.

In [ ]:
# Calculate percentage returns
daily_ret = close_prices / close_prices.shift(1) - 1
daily_ret = daily_ret.dropna()

In [ ]:
# Average daily returns
mean_daily = daily_ret.mean(axis = 0)

# Compute annualized daily returns
mean_ann = ann_factor * mean_daily

# # Compare daily and annualized expected returns (in %)
# print(np.c_[mean_daily, mean_ann] * 100)

In [ ]:
# Covariance matrix of daily returns
Sigma_daily = daily_ret.cov()

# Annualized covariance matrix
Sigma_ann = ann_factor * Sigma_daily 

In the subsequent analysis, we remove `^STOXX50E` from the analysis. We shall perform the mean-variance analysis on the constituent stocks only and check whether the `STOXX50E` is an efficient portfolio in this framework.

In [ ]:
# Extract EURO STOXX 50 information
mean_ann_index = mean_ann.iloc[0]
var_ann_index = Sigma_ann.iloc[0,0]
std_ann_index = var_ann_index ** 0.5

print(mean_ann_index, var_ann_index, std_ann_index)

In [ ]:
# Overwrite mean_ann and Sigma_ann
mean_ann = mean_ann.iloc[1:len(Symbols)]
Sigma_ann = Sigma_ann.iloc[1:len(Symbols), 1:len(Symbols)]
invSigma_ann = np.linalg.inv(Sigma_ann)

# Individual (annualized) standard deviation of returns
std_ann = pd.Series(np.diag(Sigma_ann), index=Sigma_ann.columns) ** 0.5

## Mean-Variance Analysis

### The Portfolios $\phi_a$, $\omega_{a,b}$, and $\varphi_\tau$

We first calculate $a = \mathbf{1}_d^\top \Sigma^{-1} \mathbf{1}_d$ and $b = \mathbf{1}_d \Sigma^{-1} M$, as these shall appear frequently in our calculations.

In [ ]:
# Important quantities
Vec1 = np.linspace(1, 1, len(mean_ann))
a = Vec1.T @ invSigma_ann @ Vec1
b = Vec1.T @ invSigma_ann @ mean_ann

# print(a, b)

Next, we compute the expected return and the standard deviation of the returns of the minimum-variance portfolio $\phi_a$ and the self-financing portfolio $\omega_{a,b}$, which shall define the efficient frontier $\mathscr{F}(\sigma, m)$ in the no-risky-asset case.

In [ ]:
# Properties of the minimum-variance portfolio \phi_a
phi_a = (1 / a) * invSigma_ann @ Vec1
mean_phi_a = b / a
sigma_phi_a = 1 / (a ** 0.5)

# print(mean_phi_a, sigma_phi_a)

# Properties of the self-financing portfolio \omega_{a,b}
mean_omega = np.sqrt((mean_ann - (b / a) * Vec1).T @ invSigma_ann @ (mean_ann - (b / a) * Vec1))
omega = invSigma_ann @ (mean_ann - (b / a) * Vec1) / mean_omega
sigma_omega = 1

# print(mean_omega, sigma_omega)

We then compute the expected return and the standard deviation of the returns of the tangent portfolio $\varphi_\tau$.

In [ ]:
# Properties of the tangent portfolio \varphi_\tau
mean_tau = r0 + (1 / (b - r0 * a)) * (mean_ann - r0 * Vec1).T @ invSigma_ann @ (mean_ann - r0 * Vec1)
sigma_tau = np.sqrt((1 / (b - r0 * a) ** 2) * (mean_ann - r0 * Vec1).T @ invSigma_ann @ (mean_ann - r0 * Vec1))

# print(mean_tau, sigma_tau)

### Plot $\mathscr{F}(\sigma,m)$, its asymptote, and the Capital Market Line

In [ ]:
# Define the range for the plot
range_mean_high = np.max(mean_ann) + 1
range_mean_low = np.min(mean_ann) - 1

# # Define range for the plot (including tangent and \omega portfolios)
# range_mean_high = np.max(np.array([np.max(mean_ann), mean_phi_a, mean_omega, mean_tau])) + 0.5
# range_mean_low = np.min(np.array([np.min(mean_ann), mean_phi_a, mean_omega, mean_tau])) - 0.5

# Define array of expected returns for the frontier
mean_plot = np.linspace(range_mean_low, range_mean_high, 100)

Recall that the $\mathscr{F}(\sigma, m)$, its (upper) asymptote, and the capital market line are given by the following equations given the $m$-coordinates:

\begin{align*}
    \mathscr{F}(\sigma, m): & \qquad \sigma = \sqrt{\sigma_{\phi_a}^2 + \left(\frac{m - m_{\phi_a}}{m_{\omega_{a,b}}}\right)^2} \\
    \mathscr{F}(\sigma, m) \text{ asymptote}: & \qquad \sigma = \frac{1}{m_{\omega_{a,b}}}\left(m - \frac{b}{a}\right) \\
    \text{Capital Market Line}: & \qquad \sigma = \frac{\sigma_{\varphi_\tau}}{m_{\varphi_\tau} - r_0} (m - r_0)
\end{align*}

In [ ]:
# Storage for points (\sigma) on the frontier, the asymptote, and the CML
sigma_F = pd.DataFrame()
sigma_asy = pd.DataFrame()
sigma_CML = pd.DataFrame()

# Compute \sigma for each of the three at each point
for i in range(len(mean_plot)):
    sigma_F[i] = [np.sqrt(sigma_phi_a ** 2 + ((mean_plot[i] - mean_phi_a) / mean_omega) ** 2)]
    sigma_asy[i] = [(mean_plot[i] - b / a) / mean_omega]
    sigma_CML[i] = [sigma_tau * (mean_plot[i] - r0) / (mean_tau - r0)]

In [ ]:
# Convert data frames to arrays
sigma_F = np.array(sigma_F.iloc[0])
sigma_asy = np.array(sigma_asy.iloc[0])
sigma_CML = np.array(sigma_CML.iloc[0])

In [ ]:
fig, ax = plt.subplots()

# Assign a (random) color to each stock
np.random.seed(12345)
colors = np.random.rand(len(mean_ann))

# Plot the frontier, the asymptote, and the CML
plt.plot(sigma_F, mean_plot, alpha = 1)
plt.plot(sigma_asy, mean_plot, alpha = 1)
plt.plot(sigma_CML, mean_plot, alpha = 1)

plt.legend(['Frontier', 'Asymptote', 'CML'], loc = 2)

plt.xlabel('Annualized Standard Deviation')
plt.ylabel('Annualized Expected Return')

# Add the minimum variance portfolio \phi_a
plt.scatter(sigma_phi_a, mean_phi_a, alpha = 1)
plt.annotate('MinVar', (sigma_phi_a, mean_phi_a))

# # Add the portfolio \omega
# plt.scatter(sigma_omega, mean_omega, alpha = 1)
# plt.annotate('Omega', (sigma_omega, mean_omega))

# # Add the tangent portfolio
# plt.scatter(sigma_tau, mean_tau, alpha = 1)
# plt.annotate('Tangent', (sigma_tau, mean_tau))

# Add individual stocks to the plot
plt.scatter(std_ann, mean_ann, c = colors, alpha = 0.5)

# Add the Euro Stoxx 50 to the plot and annotate
plt.scatter(std_ann_index, mean_ann_index, alpha = 1)
plt.annotate('^STOXX50E', (std_ann_index, mean_ann_index))

# Horizontal line at a target (annualized) expected return (e.g. 20%)
plt.axhline(y = 0.2, color = 'r', linestyle = '--') 

### Determine the Optimal Allocation Given a Target Expected Return

#### Optimal Allocation with Risk-Free Asset

In [ ]:
# Target (annualized) expected return
mean_target = 0.20

# Tangent portfolio risky allocation \phi_\tau
tangent_risky = (1 / (b - r0 * a)) * invSigma_ann @ (mean_ann - r0 * Vec1)

# Use Separation Theorem to compute the efficient portfolio with target expected return
sep_thm_coeff = (mean_target - r0) / (mean_tau - r0)

# Optimal allocation
opt_alloc = np.append(1 - sep_thm_coeff, sep_thm_coeff * tangent_risky)

# Mean and standard deviation
mean_opt = sep_thm_coeff * mean_tau + (1 - sep_thm_coeff) * r0   # should be mean_target, but just to check
sigma_opt = abs(sep_thm_coeff) * sigma_tau

In [ ]:
# # Check if the optimal allocation is an investment portfolio
# sum(opt_alloc)

In [ ]:
# Optimal allocation as a data frame
opt_alloc_df = pd.DataFrame({'Symbol' : np.append('Risk-Free', Symbols[0:(len(Symbols)-1)]), 
                            'Allocation' : opt_alloc})

# opt_alloc_df

In [ ]:
# Plot the frontier, the asymptote, and the CML
plt.plot(sigma_F, mean_plot, alpha = 1)
plt.plot(sigma_asy, mean_plot, alpha = 1)
plt.plot(sigma_CML, mean_plot, alpha = 1)

plt.legend(['Frontier', 'Asymptote', 'CML'], loc = 2)

plt.xlabel('Annualized Standard Deviation')
plt.ylabel('Annualized Expected Return')

# Add the efficient portfolio calculated in the previous step
plt.scatter(sigma_opt, mean_opt, alpha = 1)
plt.annotate('Efficient', (sigma_opt, mean_opt))

# Add the minimum variance portfolio \phi_a
plt.scatter(sigma_phi_a, mean_phi_a, alpha = 1)
plt.annotate('MinVar', (sigma_phi_a, mean_phi_a))

# Add individual stocks to the plot
plt.scatter(std_ann, mean_ann, c = colors, alpha = 0.5)

# Add the Euro Stoxx 50 to the plot and annotate
plt.scatter(std_ann_index, mean_ann_index, alpha = 1)
plt.annotate('^STOXX50E', (std_ann_index, mean_ann_index))

# Horizontal line at a target (annualized) expected return (e.g. 20%)
plt.axhline(y = 0.2, color = 'r', linestyle = '--') 

As expected, this efficient portfolio is located on the capital market line, as it contains an allocation to the risk-free asset.

#### Optimal Allocation without a Risk-Free Asset

To calculate the optimal allocation without a risk-free asset, recall that portfolios in $\mathscr{F}$ have the representation 

$$\phi = \phi_a + \lambda \omega_{a,b}$$

for some $\lambda \in \mathbb{R}$. To determine the appropriate value of $\lambda$ to achieve a target expected return $m_\phi = \bar{m}$, we note that, in Proposition 3.9, we have $m_\phi = m_{\phi_a} + \lambda m_{\omega_{a,b}}$. This implies that the $\lambda$ associated to the target expected return $\bar{m}$ is $\lambda = (\bar{m} - m_{\phi_a}) / m_{\omega_{a,b}}$. This implies that the optimal investment strategy to achieve the target expected return $\bar{m}$ is

$$\phi^* = \phi_a + \frac{\bar{m} - m_{\phi_a}}{m_{\omega_{a,b}}} \omega_{a,b}.$$

In [ ]:
# Optimal allocation with no risk-free asset
opt_alloc_nrf = phi_a + ((mean_target - mean_phi_a) / mean_omega) * omega

# Mean and standard deviation of the optimal allocation
mean_opt_nrf = mean_target
sigma_opt_nrf = np.sqrt(sigma_phi_a ** 2 + ((mean_opt_nrf - mean_phi_a) / mean_omega) ** 2)

In [ ]:
# # Check if the optimal allocation is an investment portfolio
# sum(opt_alloc_nrf)   # Should be equal to 1

# # Check standard deviation of optimal portfolio return
# np.sqrt(opt_alloc_nrf.T @ Sigma_ann @ opt_alloc_nrf), sigma_opt_nrf  # Should be equal

In [ ]:
# Optimal allocation (no risk-free) as a data frame
opt_alloc_nrf_df = pd.DataFrame({'Symbol' : Symbols[0:(len(Symbols)-1)], 
                            'Allocation' : opt_alloc_nrf})

# opt_alloc_nrf_df

In [ ]:
# Plot the frontier, the asymptote, and the CML
plt.plot(sigma_F, mean_plot, alpha = 1)
plt.plot(sigma_asy, mean_plot, alpha = 1)
plt.plot(sigma_CML, mean_plot, alpha = 1)

plt.legend(['Frontier', 'Asymptote', 'CML'], loc = 2)

plt.xlabel('Annualized Standard Deviation')
plt.ylabel('Annualized Expected Return')

# Add the efficient portfolio with risk-free asset calculated in the previous step
plt.scatter(sigma_opt, mean_opt, alpha = 1)
plt.annotate('Efficient', (sigma_opt, mean_opt + 0.05))

# Add the efficient portfolio with no risk-free asset calculated in the previous step
plt.scatter(sigma_opt_nrf, mean_opt_nrf, alpha = 1)
plt.annotate('Efficient (NRF)', (sigma_opt_nrf + 0.01, mean_opt_nrf))

# Add the minimum variance portfolio \phi_a
plt.scatter(sigma_phi_a, mean_phi_a, alpha = 1)
plt.annotate('MinVar', (sigma_phi_a, mean_phi_a))

# # Add individual stocks to the plot
# plt.scatter(std_ann, mean_ann, c = colors, alpha = 0.5)

# Add the Euro Stoxx 50 to the plot and annotate
plt.scatter(std_ann_index, mean_ann_index, alpha = 1)
plt.annotate('STOXX50E', (std_ann_index, mean_ann_index))

# Horizontal line at a target (annualized) expected return (e.g. 20%)
plt.axhline(y = 0.2, color = 'r', linestyle = '--') 

## Tracking Error Variance Analysis

Using the Euro Stoxx 50 index as the benchmark portfolio, we seek an investment portfolio that minimizes the tracking error variance with respect to the benchmark.

### Estimate the Betas w.r.t. the Benchmark

To conduct this analysis, we first need to estimate the beta of each stock with respect to the Euro Stoxx 50 index. We employ a "historical simulation" approach similar to what we used in the mean-variance analysis. The output of the following codes is the vector $\beta_B$ whose $i$th entry is the beta of the $i$th stock with respect to the benchmark, $i=1,\dots,d$.

In [ ]:
# Annualize the daily returns
daily_ret_ann = ann_factor * daily_ret
daily_ret_ann = daily_ret_ann.dropna()   # Remove NaNs (needed for regression)

# Compute the excess returns of the benchmark
exc_ret_benchmark = (daily_ret_ann['^STOXX50E'] - r0).array

# Storage for estimated betas and R-squared
stock_betas = pd.DataFrame()
stock_rsq = pd.DataFrame()

# Iterate beta estimation over each stock
index_set_excl_benchmark = daily_ret_ann.columns[0:(len(daily_ret_ann.columns)-1)]

for i in index_set_excl_benchmark:
    exc_ret_stock = (daily_ret_ann[i] - r0).array
    reg = LinearRegression(fit_intercept = False).fit(exc_ret_benchmark.reshape(-1, 1), exc_ret_stock)
    beta = reg.coef_[0]
    rsq = reg.score(exc_ret_benchmark.reshape(-1, 1), exc_ret_stock)
    stock_betas[i] = [beta]
    stock_rsq[i] = [rsq]
    
# Write the stock betas as a column vector
stock_betas = stock_betas.T
stock_betas.columns = ["Betas"]

Alternatively, the stock betas with respect to the index can be calculated from the covariance matrix of returns using the formula

$$\beta_{B,i} = \frac{\mathsf{Cov}(\mathcal{R}(\phi_B), R^i)}{\sigma_B^2}$$

where $\mathcal{R}(\phi_B)$ is the (random) return of the benchmark index/asset, $R^i$ is the (random) return of the $i$th asset, and $\sigma_B^2 := \mathsf{Var}(\mathcal{R}(\phi_B))$ is the variance of the return of the benchmark index/asset. The empirical covariance matrix can be used as estimates for the required quantities to calculate the $i$th asset beta.

Note that the results from this procedure will not perfectly match with those from the regression method, due to some data pre-processing steps involved in the latter method.

In [ ]:
# Stock betas using covariance matrix
stock_betas2 = daily_ret.cov()['^STOXX50E'] / daily_ret.cov().loc['^STOXX50E', '^STOXX50E']

### Estimate Auxilliary Quantities

In this section, we define the following matrices:

\begin{align*}
\mathbf{B} & := [M, \mathbf{1}_d] \in \mathbb{R}^{d\times 2} \\
\mathbf{Q} & := \mathbf{B}_\perp (\mathbf{B}_\perp^\top \Sigma \mathbf{B}_\perp)^{-1} \mathbf{B}_\perp^\top \in \mathbb{R}^{d \times d} \\
(\mathbf{B}^\top)^\mathscr{r} & := \mathbf{B} (\mathbf{B}^\top \mathbf{B})^{-1} \in \mathbb{R}^{d \times 2}\\
\mathbf{C} & := (\mathbf{I}_d - \mathbf{Q} \Sigma) (\mathbf{B}^\top)^\mathscr{r} \in \mathbb{R}^{d \times 2}
\end{align*}

which are needed to characterize the TEV optimal portfolios 

$$\phi^* = \sigma_B^2 \mathbf{Q} \beta_B + \mathbf{C} \left[\begin{array}{c} \bar{m} \\ 1 \end{array}\right], \qquad \bar{m} \in \mathbb{R}$$

and the TEV efficient frontier $\mathscr{T}(\sigma, m)$.

In Python, we can compute $\mathbf{B}_\perp$ using the ``null_space`` function in ``scipy.linalg``.

In [ ]:
# Define the matrix B
B_mat = np.array([mean_ann, Vec1]).T

In [ ]:
# Load null_space function from scipy.linalg
from scipy.linalg import null_space

# Compute B_\perp
B_perp = null_space(B_mat.T)

In [ ]:
# Compute inverse term in the matrix Q
inv_mat_term = np.linalg.inv(B_perp.T @ Sigma_ann.to_numpy() @ B_perp)

# Compute the matrix Q
Q_mat = B_perp @ inv_mat_term @ B_perp.T

In [ ]:
# Compute right-inverse of B^\top (from definition)
BT_rinv = B_mat @ np.linalg.inv(B_mat.T @ B_mat)

# Compute the matrix C
C_mat = (np.identity(len(mean_ann)) - Q_mat @ Sigma_ann.to_numpy()) @ BT_rinv

### Determine TEV-Optimal Portfolio for Given Target Mean

Recall: we have set a target expected (annualized) return of $\bar{m} = 0.20$.

In [ ]:
# TEV-optimal portfolio
opt_alloc_tev = var_ann_index * Q_mat @ stock_betas.to_numpy() + C_mat @ np.array([[mean_target], [1]])

# Check if constraints are satisfied
opt_alloc_tev.T @ mean_ann, opt_alloc_tev.T @ Vec1    # Should be equal to (0.2, 1)

In [ ]:
# Determine the mean and standard deviation of the porfolio return of the TEV-optimal portfolio
sigma_opt_tev = np.sqrt(opt_alloc_tev.T @ Sigma_ann.to_numpy() @ opt_alloc_tev)
mean_opt_tev = opt_alloc_tev.T @ mean_ann   # should be equal to target mean

In [ ]:
sigma_opt_tev

### TEV-Optimal Portfolio (Function)

In [ ]:
# Tracking error variance (TEV) optimal portfolios

def tev_optim(target_mean, mean_return_vector, return_covariance_matrix, asset_betas):

    """
    Determines the composition of the tracking error variance optimal portfolio with respect to a specified benchmark asset 
    for the given target mean.

    usage:
        tev_optimal_portfolio = tev_optim(target_mean, mean_return_vector, return_covariance_matrix, asset_betas)

    inputs:
        target_mean               : target expected (annualized) return of the mean-variance optimal portfolio
        mean_return_vector        : mean of returns of the risky assets (annualized)
        return_covariance_matrix  : covariance of the returns of the risky assets (annualized)
        asset_betas               : betas of the risky assets with respect to the benchmark asset

    outputs:
        tev_optimal_portfolio     : composition of the TEV-optimal (investment) portfolio satisfying the target mean return 
    """

    # Introduce notation
    mean_ann = mean_return_vector
    Sigma_ann = return_covariance_matrix

    # Calculate auxiliary quantities to characterize TEV optimal portfolios

    ## - The matrix B and its orthogonal complement
    B_mat = np.array([mean_ann, Vec1]).T
    B_perp = sp.linalg.null_space(B_mat.T)

    ## - Inverse matrix in the matrix Q
    inv_mat_term = np.linalg.inv(B_perp.T @ Sigma_ann.to_numpy() @ B_perp)

    ## - The matrix Q
    Q_mat = B_perp @ inv_mat_term @ B_perp.T

    ## - Right inverse of B^T
    BT_rinv = B_mat @ np.linalg.inv(B_mat.T @ B_mat)

    ## - Compute the matrix C
    C_mat = (np.identity(len(mean_ann)) - Q_mat @ Sigma_ann.to_numpy()) @ BT_rinv

    # Composition of the TEV-optimal portfolio
    opt_alloc_tev = var_ann_index * Q_mat @ asset_betas.to_numpy() + C_mat @ np.array([[target_mean], [1]])

    return(opt_alloc_tev)

    ## END

In [ ]:
# Test the function
tev_optimal_portfolio = tev_optim(target_mean = mean_target, 
                                  mean_return_vector = mean_ann, 
                                  return_covariance_matrix = Sigma_ann, 
                                  asset_betas = stock_betas)

## - Verify target mean and investment portfolio constraint
print(tev_optimal_portfolio.T @ mean_ann)
print(tev_optimal_portfolio.T @ Vec1)

In [ ]:
## - Check TEV-optimal portfolio volatility
np.sqrt(tev_optimal_portfolio.T @ Sigma_ann @ tev_optimal_portfolio)

### Plot the TEV optimal portfolio on the risk-return space

In [ ]:
# Plot the frontier, the asymptote, and the CML
plt.plot(sigma_F, mean_plot, alpha = 1)
plt.plot(sigma_asy, mean_plot, alpha = 1)
plt.plot(sigma_CML, mean_plot, alpha = 1)

plt.legend(['Frontier', 'Asymptote', 'CML'], loc = 2)

plt.xlabel('Annualized Standard Deviation')
plt.ylabel('Annualized Expected Return')

# Add the efficient portfolio with risk-free asset calculated in the previous step
plt.scatter(sigma_opt, mean_opt, alpha = 1)
plt.annotate('WRF', (sigma_opt, mean_opt + 0.05))

# Add the efficient portfolio with no risk-free asset calculated in the previous step
plt.scatter(sigma_opt_nrf, mean_opt_nrf, alpha = 1)
plt.annotate('NRF', (sigma_opt_nrf + 0.01, mean_opt_nrf))

# Add the TEV-optimal portfolio
plt.scatter(sigma_opt_tev, mean_opt_tev, alpha = 1)
plt.annotate('TEV', (sigma_opt_tev, mean_opt_tev))

# Add the minimum variance portfolio \phi_a
plt.scatter(sigma_phi_a, mean_phi_a, alpha = 1)
plt.annotate('MinVar', (sigma_phi_a, mean_phi_a))

# Add the Euro Stoxx 50 to the plot and annotate
plt.scatter(std_ann_index, mean_ann_index, alpha = 1)
plt.annotate('STOXX50E', (std_ann_index, mean_ann_index))

# Horizontal line at a target (annualized) expected return (e.g. 20%)
plt.axhline(y = 0.2, color = 'r', linestyle = '--') 

In [ ]:
# Calculate the beta of the TEV-optimal portfolio
beta_TEV = opt_alloc_tev.T @ stock_betas
beta_TEV

### Construct the TEV efficient frontier

The TEV efficient frontier is obtained via a lateral shift of the MV efficient frontier (with no risk-free asset) $\mathscr{F}(\sigma, m)$: for each $(\sigma', m') \in \mathscr{F}(\sigma, m)$, the corresponding point on $\mathscr{T}(\sigma, m)$ with the same $m$-coordinate $m_{\mathsf{TEV}} = m'$ has a $\sigma$-coordinate

$$\sigma_{\mathsf{TEV}} = \sqrt{(\sigma')^2 + \sigma_B^4 \beta_B^\top \mathbf{Q} \beta_B}.$$

In [ ]:
# Calculate the sigma-coordinates of the TEV frontier
sigma_TEV = np.sqrt(sigma_F ** 2 + (var_ann_index ** 2) * stock_betas.to_numpy().T @ Q_mat @ stock_betas.to_numpy())
sigma_TEV = sigma_TEV.T

In [ ]:
# Plot the frontier, the asymptote, and the CML
plt.plot(sigma_F, mean_plot, alpha = 1)
plt.plot(sigma_asy, mean_plot, alpha = 1)
plt.plot(sigma_CML, mean_plot, alpha = 1)
plt.plot(sigma_TEV, mean_plot, alpha = 1)

plt.legend(['MV Frontier', 'Asymptote', 'CML', 'TEV Frontier'], loc = 2)

plt.xlabel('Annualized Standard Deviation')
plt.ylabel('Annualized Expected Return')

# Add the efficient portfolio with risk-free asset calculated in the previous step
plt.scatter(sigma_opt, mean_opt, alpha = 1)
plt.annotate('WRF', (sigma_opt, mean_opt + 0.05))

# Add the efficient portfolio with no risk-free asset calculated in the previous step
plt.scatter(sigma_opt_nrf, mean_opt_nrf, alpha = 1)
plt.annotate('NRF', (sigma_opt_nrf + 0.01, mean_opt_nrf))

# Add the TEV-optimal portfolio
plt.scatter(sigma_opt_tev, mean_opt_tev, alpha = 1)
plt.annotate('TEV', (sigma_opt_tev, mean_opt_tev))

# Add the minimum variance portfolio \phi_a
plt.scatter(sigma_phi_a, mean_phi_a, alpha = 1)
plt.annotate('MinVar', (sigma_phi_a, mean_phi_a))

# Add the Euro Stoxx 50 to the plot and annotate
plt.scatter(std_ann_index, mean_ann_index, alpha = 1)
plt.annotate('STOXX50E', (std_ann_index, mean_ann_index))

# Horizontal line at a target (annualized) expected return (e.g. 20%)
plt.axhline(y = 0.2, color = 'r', linestyle = '--', alpha = 0.5) 
plt.axhline(y = mean_ann_index, color = 'b', linestyle = '--', alpha = 0.5)
plt.axhline(y = r0, color = 'g', linestyle = '--', alpha = 0.5)
plt.axvline(x = sigma_opt_tev, color = 'r', linestyle = '--', alpha = 0.5)

### Some Observations (Roll, 1992)

- The benchmark (Euro Stoxx 50) is MV-inefficient in the sense that it is located to the right of the MV efficient frontier. 

- The TEV-optimal portfolio is also MV-inefficient. In this financial market, regardless of the target expected return $\bar{m}$, the TEV-optimal portfolio is always MV-inefficient.

- There exist feasible portfolios (portfolios on or to the right of the MV-efficient frontier) with higher expected returns and lower risk/standard deviation compared to the TEV-optimal portfolio and the benchmark. These portfolios are said to *dominate* the TEV-optimal portfolio and the benchmark.
    
    - The portfolios that dominate the TEV-optimal portfolio are whose whose expected return and risk are located in the region bounded above by the MV efficient frontier, bounded below by the line $m = \bar{m} = m_{\phi^*}$, and bounded to the right by the line $\sigma = \sigma_{\phi^*}$.
    
- The TEV-optimal portfolio has a beta higher than 1 with respect to the benchmark if $m_B > m_{\phi_a}$. Furthermore, if $m_B > m_{\phi_a}$, then the TEV-optimal portfolio does not dominate the benchmark.